# This is an attempt to build a testing battery.

In [2]:
import sys
import statistics
from itertools import product
from timeit import default_timer as timer
from tqdm import tqdm
sys.path.append("/home/jackson/Research/GillesPy2/")
import gillespy2
from scipy import stats
import numpy

In [3]:
#Doing this in known order of good solvers. I imagine I'm going to have to nudge some stuff.
#Basic ODE Is messed up. I think I remember looking into this before and it not being super difficult.
#Linting the Optimized SSA Solver stuff, coercing into standard naming pattern. (I consider Basic SSA the standard.)
#Ask about Dalton?
#C stuff does not work with my plotting function! It uses a list and not a dictionary. Hooray.....
#Removed extraneous imports from tau_leaping basic.
#Basic_Tau_Leaping doesn't work. I know it's status is temptuous.
#Regular Tau Hangs (I threw 30 minutes at it for the MM and TCE model, which are generally instataneous)
#Final List before going to research meeting is SSA, Root, and Optimized
from gillespy2.basic_ssa_solver import BasicSSASolver
from gillespy2.ssa_c_solver import SSACSolver

In [5]:
class MichaelisMenten(gillespy2.Model):
     def __init__(self, parameter_values=None):
            #initialize Model
            gillespy2.Model.__init__(self, name="Michaelis_Menten")
            
            #parameters
            rate1 = gillespy2.Parameter(name='rate1', expression= 0.0017)
            rate2 = gillespy2.Parameter(name='rate2', expression= 0.0001)
            rate3 = gillespy2.Parameter(name='rate3', expression = 0.1)
            self.add_parameter([rate1,rate2,rate3])
            
            #Species
            A = gillespy2.Species(name='A', initial_value=301)
            B = gillespy2.Species(name='B', initial_value=120)
            C = gillespy2.Species(name='C', initial_value=0)
            D = gillespy2.Species(name='D', initial_value=0)
            self.add_species([A, B, C, D])
            
            #reactions
            r1 = gillespy2.Reaction(name="r1",reactants={A:1,B:1}, products={C:1},
                   rate=rate1)
            
            r2 = gillespy2.Reaction(name="r2",reactants={C:1}, products={A:1,B:1},
                    rate=rate2)
            
            r3 = gillespy2.Reaction(name="r3",reactants={C:1}, products={B:1,D:1},
                    rate=rate3)
            self.add_reaction([r1,r2,r3])
            self.timespan(numpy.linspace(0,100,101))

## Test Importation of Models.

In [6]:
modelList = [MichaelisMenten()]

solverList = [BasicSSASolver, SSACSolver]
timingList = []


for model, solver in product(modelList, solverList):
    medianList = []
    exterior_stats = []
    for i in tqdm(range(100), desc = f'Model: {model.name}, Solver: {solver}'):
        standard_results = model.run(stochkit_home="/home/jackson/StochKit/StochKit")
        start = timer()
        test_results =model.run(solver=solver)
        stop = timer()
        medianList.append(stop-start)
        interior_stats = []
        for species in standard_results[0].keys():
            if solver == SSACSolver:
                interior_stats.append(stats.ks_2samp(standard_results[0][species], test_results[0][species])[0])
            else:
                interior_stats.append(stats.ks_2samp(standard_results[0][species], test_results[species])[0])
        exterior_stats.append(statistics.median(interior_stats))
    median = statistics.median(medianList)
    timingList.append([model, solver, median])

Model: Michaelis_Menten, Solver: <class 'gillespy2.basic_ssa_solver.BasicSSASolver'>: 100%|██████████| 100/100 [00:04<00:00, 24.01it/s]
Model: Michaelis_Menten, Solver: <class 'gillespy2.ssa_c_solver.SSACSolver'>: 100%|██████████| 100/100 [00:49<00:00,  2.02it/s]


In [5]:
print(*exterior_stats)

0.5900000000000001 0.5641584158415841 0.6302970297029702 0.6335643564356436 0.6102970297029703 0.5741584158415841 0.5502970297029703 0.5939603960396039 0.5641584158415841 0.6 0.5800990099009902 0.6335643564356436 0.5501980198019802 0.584059405940594 0.6236633663366337 0.6038613861386138 0.5741584158415841 0.5939603960396039 0.65 0.6103960396039604 0.6236633663366337 0.5938613861386138 0.6038613861386138 0.6236633663366337 0.5939603960396039 0.5738613861386138 0.6002970297029703 0.6100990099009901 0.6601980198019801 0.6304950495049504 0.6200990099009901 0.62 0.62 0.6435643564356436 0.5601980198019803 0.6134653465346535 0.5702970297029702 0.61 0.6100990099009901 0.6232673267326733 0.6100990099009901 0.5900000000000001 0.5902970297029703 0.65 0.6401980198019801 0.5542574257425742 0.5601980198019803 0.6335643564356436 0.6335643564356436 0.5900990099009901 0.61 0.61 0.5838613861386138 0.584059405940594 0.5800990099009902 0.64 0.6201980198019802 0.6 0.5904950495049505 0.6301980198019802 0.63

In [6]:
from gillespy2.example_models import LacOperon
model = LacOperon()
start = timer()
results = model.run(solver=BasicSSASolver)
model.plot(results)

ModuleNotFoundError: No module named 'gillespy2.example_models'

In [ ]:
#Workaround for tqdm doing some weird dynamic scaling that causes a runtime error in JupyterLab.
#Operation seems to take to long for Jupyter Lab when evaluating LacOperon Model for any solvers when coerced into a function list. Individual Stuff works well.
for x in tqdm(range(len(solverList))):
    model = LacOperon()
    start = timer()
    model.run(solver=solverList[x])
    stop = timer()
    timingList.append([LacOperon, str(solverList[x]), stop-start])

In [ ]:
from gillespy2.example_models import MichaelisMenten
import h5py
model = MichaelisMenten()
standard_results = model.run(stochkit_home="/home/jackson/StochKit/StochKit")
print(standard_results)
# with h5py.File('MichaelisMenten.hdf5','w') as f:
for species in model.listOfSpecies.keys():
     print(len(standard_results[0][f'{species}']))